In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
order_base=spark.sparkContext.textFile('/public/trendytech/orders/orders_1gb.csv')

In [3]:
order_rdd=order_base.map(lambda x:(x.split(",")[2],x.split(",")[3]))

In [4]:
cust_base=spark.sparkContext.textFile('/public/trendytech/retail_db/customers/part-00000')

In [5]:
cust_rdd=cust_base.map(lambda x:(x.split(",")[0],x.split(",")[8]))

In [6]:
order_rdd.take(5)

[('11599', 'CLOSED'),
 ('256', 'PENDING_PAYMENT'),
 ('12111', 'COMPLETE'),
 ('8827', 'CLOSED'),
 ('11318', 'COMPLETE')]

In [7]:
cust_rdd.take(5)

[('1', '78521'),
 ('2', '80126'),
 ('3', '00725'),
 ('4', '92069'),
 ('5', '00725')]

In [8]:
customer_broadcast=spark.sparkContext.broadcast(cust_rdd.collect())    #####Broadcasted

In [9]:
def get_pin(customer_id):
    try:
        return customer_broadcast.value[customer_id]
    except:
        return "-1"

In [10]:
joined_rdd=order_rdd.map(lambda x:(get_pin(int(x[0])),x[1]))

In [11]:
joined_rdd.saveAsTextFile("data/broadcast_joined_data")